In [122]:
#import Libriaries
import xml.etree.ElementTree as ET
import pandas as pd
import re
import pyodbc #oldwayto connect to sql server
import pydash #to format table and field names
import pandasgui as pg #to visualize dataframes
import sqlalchemy  #new way to connect to sql server

In [123]:
# Define the connection details
server = 'jacobo-dev.database.windows.net'
port = '1433'
database = 'jacobo-dev-sqlserver-azure-001'
username = 'azure-admin'
password = 'ja-2023-un0ypzjo'
driver = '{ODBC Driver 18 for SQL Server}'

# Define the connection string
conn_str = f"DRIVER={driver};SERVER={server},{port};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30"

In [124]:
# Test SQL DB Connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")
    conn.close()
except pyodbc.Error as e:
    print("Error connecting to database:", e)


Error connecting to database: ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Unable to complete login process due to delay in login response (258)')


In [125]:
#function to get all the child elements of a given xml element

def get_child_elements(xml_root):
    # Create a list to store all child elements
    elements = []
    #append xml root to elements list
    #elements.append(xml_root)
    # Get all child elements of a given xml element
    for child in xml_root:
        elements.append(child)
        elements.extend(get_child_elements(child))
    return elements


In [126]:
def process_xml_elements(conn, elements):
    """
    Inserts the attributes of each XML element into a SQL table with the same name
    as the element tag.
    """
    #conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    # loop through the elements and their attributes
    for element in elements:
        # create the SQL table name using the element tag
        table_name = element.tag

        # create the SQL create table query
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
        for attr in element.attrib:
            create_table_query += f"{attr} VARCHAR(MAX), "
        create_table_query = create_table_query[:-2] + ")"

        # execute the create table query
        cursor.execute(create_table_query)
        cursor.commit()

        # insert the element attributes into the table
        insert_query = f"INSERT INTO {table_name} ("
        for attr in element.attrib:
            insert_query += f"{attr}, "
        insert_query = insert_query[:-2] + ") VALUES ("
        for attr in element.attrib:
            insert_query += f"'{element.attrib[attr]}', "
        insert_query = insert_query[:-2] + ")"

        # execute the insert query
        cursor.execute(insert_query)
        cursor.commit()

    # close the database connection
    conn.close()

In [127]:
def pascal_case(text):
    return ''.join(word.capitalize() for word in text.split())


In [128]:
# Define file to read
xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Nomina\EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml"
xml_tree = ET.parse(xml_file_path)
xml_root = xml_tree.getroot()

In [129]:
#Read XML Child elements
elements = get_child_elements(xml_root)

#add xml root to read it with their child elements
elements.append(xml_root)

#print child elements
for element in elements:
    if element.attrib:
        print(element.tag, element.attrib)

{http://www.sat.gob.mx/cfd/3}Emisor {'Rfc': 'EMS2103108P3', 'Nombre': 'ESAB MEXICO SERVICES SA DE CV', 'RegimenFiscal': '601'}
{http://www.sat.gob.mx/cfd/3}Receptor {'Rfc': 'AOAR951019842', 'Nombre': 'Roberto Carlos Alonso Alvarez', 'UsoCFDI': 'P01'}
{http://www.sat.gob.mx/cfd/3}Concepto {'ClaveProdServ': '84111505', 'Cantidad': '1', 'ClaveUnidad': 'ACT', 'Descripcion': 'Pago de nómina', 'ValorUnitario': '43376.49', 'Importe': '43376.49', 'Descuento': '14787.52'}
{http://www.sat.gob.mx/nomina12}Nomina {'Version': '1.2', 'TipoNomina': 'O', 'FechaPago': '2022-08-15', 'FechaInicialPago': '2022-08-01', 'FechaFinalPago': '2022-08-15', 'NumDiasPagados': '15.000', 'TotalPercepciones': '43376.49', 'TotalDeducciones': '14787.52', 'TotalOtrosPagos': '0.00'}
{http://www.sat.gob.mx/nomina12}Emisor {'RegistroPatronal': 'Y3936393101'}
{http://www.sat.gob.mx/nomina12}Receptor {'Curp': 'AOAR951019HNLLLB06', 'NumSeguridadSocial': '43129510616', 'FechaInicioRelLaboral': '2022-02-01', 'Antigüedad': 'P28W

**SQL Query with pandas.**

In [131]:
## Test Query Retrieve Data

# Define the SQL query
sample_query = "SELECT * FROM [cfditest].[SampleTable]"

# Establish the database connection
conn = pyodbc.connect(conn_str)

# Execute the SQL query and load the data into a pandas dataframe
df = pd.read_sql(sample_query, conn)

# Print the first five rows of the dataframe
print(df.head())

# display data using pandasgui
pg.show(df.head())

C:\Users\Roberto\AppData\Local\Temp\ipykernel_27424\2157251554.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


   Id         Name  Age                    Email
0   1     John Doe   30     john.doe@example.com
1   2   Jane Smith   25   jane.smith@example.com
2   3  Bob Johnson   40  bob.johnson@example.com
3   4    Alice Lee   35    alice.lee@example.com
4   5    Tom Brown   28    tom.brown@example.com


In [ ]:
#cleantag
for element in elements:
    cleaned_tag = re.sub(r"{.*?}", "", element.tag)
    print(cleaned_tag)